In [8]:
%load_ext autoreload
%autoreload 2

In [9]:
# Libs
import logging
import pandas as pd
from collections import OrderedDict
import pickle
import torch

# Own Modules
from source_torch.mlca.mlca import mlca_mechanism
from source_torch.mlca.mlca_setup import set_value_model_parameters
#from source.mlca.mlca import mlca_mechanism
#from source.mlca.mlca_setup import set_value_model_parameters

import datetime
import os
from source_torch.util import save_result, load_result
import numpy as np

In [10]:
import GPUtil
GPUtil.showUtilization()

| ID | GPU  | MEM |
-------------------
|  0 |  31% | 12% |
|  1 |   0% |  0% |
|  2 |   0% |  0% |
|  3 |   0% |  0% |
|  4 |   0% |  0% |
|  5 | 100% | 97% |
|  6 |   0% | 97% |
|  7 |   0% |  0% |


In [11]:
torch.cuda.set_device(1)
torch.cuda.is_available()
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device

True

In [16]:
iterations = 4
seed = 6
while iterations > 0:
    x = datetime.datetime.now()
    m = x.month
    d = x.day
    h = x.hour
    mi = x.minute

    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)
    # log debug to console
    #logging.basicConfig(level=logging.DEBUG, format='%(asctime)s:               %(message)s', datefmt='%H:%M:%S')
    logging.basicConfig(level=logging.WARNING, format='%(asctime)s:               %(message)s', datefmt='%H:%M:%S')
    #logging.basicConfig(level=logging.INFO, format='%(asctime)s:               %(message)s', datefmt='%H:%M:%S')    
    
    configdict_mlca = OrderedDict([('SATS_domain_name','GSVM'),
                        ('SATS_auction_instance_seed', seed),
                        ('bidders',0),
                        ('items',0),
                        ('bidder_ids',0),
                        ('Qinit', 30),
                        ('Qmax', 50),#50
                        ('Qround',0),
                        ('Starter','mlca_extra'),
                        ('epochs' , 512),#512
                        ('batch_size' , 32),
                        ('regularization_type' , 'l1_l2'),
                        ('layer_N' , [10, 10]),
                        ('layer_R' ,[32, 32] ),
                        ('layer_L' , [16, 16]),
                        ('NN_parameters',[]),
                        ('bigM',6000),
                        ('warm_start',False),
                        ('MIP_parameters',[]),
                        ('scaler',None),
                        ('init_bids_and_fitted_scaler',[None,None]),
                        ('return_allocation',True),
                        ('return_payments',True),
                        ('lra',0.01),
                        ('regularization',1e-5),
                        ('calc_efficiency_per_iteration',True),
                        ('active_learning_algorithm', 'galo'),
                        ('presampled_n', 20),
                        ('presampled_algorithm', 'unif')])

    configdict_mlca = set_value_model_parameters(configdict_mlca)

    res = mlca_mechanism(configdict = configdict_mlca)

    seed += 1
    iterations -= 1

    if res == 'UNIQUENESS CHECK FAILED see logfile':
        # add one more iteration to the loop
        iterations += 1
        continue
    dirname = './experiments/MLCA/Torch/results/' + configdict_mlca['SATS_domain_name'] +'_'+ configdict_mlca['active_learning_algorithm']
    result_dir = './experiments/MLCA/Torch/results/' + configdict_mlca['SATS_domain_name'] +'_' + configdict_mlca['active_learning_algorithm'] +'/' +str(m)+'_'+str(d) + '_' + str(h) + '_' + str(mi) + '_' + str(configdict_mlca['SATS_auction_instance_seed']-1)
    os.makedirs(dirname,exist_ok=True)
    save_result(result_dir, res)

16:03:23:               START MLCA:
16:03:23:               -----------------------------------------------
16:03:23:               Model: GSVM
16:03:23:               Seed SATS Instance: 6
16:03:23:               Qinit: 30
16:03:23:               Qmax: 50
16:03:23:               Qround: 7




------------------------ SATS parameters ------------------------
Value Model:  GSVM
Number of Bidders:  7
Number of BidderTypes:  2
Number of Items:  18
Scaler:  None

------------------------ DNN  parameters ------------------------
Epochs: 512
Batch Size: 32
Regularization: l1_l2

Bidder_0:
regularization: 1e-05
learning_rate: 0.01
architecture: [32, 32]
dropout: True
dropout_prob: 0.05
epochs: 512
batch_size: 32
regularization_type: l1_l2

Bidder_1:
regularization: 1e-05
learning_rate: 0.01
architecture: [32, 32]
dropout: True
dropout_prob: 0.05
epochs: 512
batch_size: 32
regularization_type: l1_l2

Bidder_2:
regularization: 1e-05
learning_rate: 0.01
architecture: [32, 32]
dropout: True
dropout_prob: 0.05
epochs: 512
batch_size: 32
regularization_type: l1_l2

Bidder_3:
regularization: 1e-05
learning_rate: 0.01
architecture: [32, 32]
dropout: True
dropout_prob: 0.05
epochs: 512
batch_size: 32
regularization_type: l1_l2

Bidder_4:
regularization: 1e-05
learning_rate: 0.01
architectu

DOcplexException: Model<sampling> did not solve successfully

In [16]:
# import numpy as np
# np.mean(scores), np.std(scores)
#print(res.keys())
#len(res['Initial_Bundles']['Bidder_2'][0])
# for bundles in res['Initial_Bundles']:
#     print(bundles)

#print(len(res['Elicited Bids']['Bidder_1'][0]))


result_dir = './experiments/MLCA/Torch/results/GSVM' +'_' + 'gali' +'_' +str(6)+'_'+str(29) + '_' + str(17) + '_' + str(0) + '/results'

In [17]:
results = load_result(result_dir)

./experiments/MLCA/Torch/results/LSVM_gali_6_29_17_0/results is loaded 


In [20]:
# check if all vectors are unique in  results[0]['Elicited Bids']['Bidder_1'][0]
results[0]['MLCA Efficiency']

0.8780602421764311

In [17]:
results[0]['Statistics']['Efficiency per Iteration'] 

OrderedDict([(0, 0.8122982289500839),
             (1, 0.992808946494547),
             (2, 0.992808946494547)])

In [25]:
dirname_active_lsvm = './experiments/MLCA/Torch/results'+'19'+'_'+'6' + '_' + '17' + '_' + '7'
dirname_active_mrvm = './experiments/MLCA/Torch/results'+'6'+'_'+'20' + '_' + '12' + '_' + '56'
dirname_active_gsvm = './experiments/MLCA/Torch/results'+'6'+'_'+'20' + '_' + '18' + '_' + '25'
dirname_unif_lsvm = './experiments/MLCA/Torch/results'+'6'+'_'+'20' + '_' + '19' + '_' + '51'
loaded_res_1 = load_result(dirname_active_lsvm)
loaded_res_2 = load_result(dirname_active_mrvm)
loaded_res_3 = load_result(dirname_active_gsvm)
loaded_res_4 = load_result(dirname_unif_lsvm)
results_all = [loaded_res_1, loaded_res_2, loaded_res_3, loaded_res_4]

./experiments/MLCA/Torch/results19_6_17_7 is loaded 
./experiments/MLCA/Torch/results6_20_12_56 is loaded 
./experiments/MLCA/Torch/results6_20_18_25 is loaded 
./experiments/MLCA/Torch/results6_20_19_51 is loaded 


In [8]:
loaded_res_1[0]
#loaded_res[0]['MLCA Allocation']
# for res_l in loaded_res:
#     try:
#         print(res_l['MLCA Efficiency'])
#     except:
#         pass
# loaded_res[0]['MLCA Efficiency']

NameError: name 'loaded_res_1' is not defined

In [32]:
loaded_res[0]['Statistics']['Efficiency per Iteration']

OrderedDict([(0, 0.6880173599333909), (1, 0.8845542910694237)])

In [20]:
loaded_res[0]['Statistics']['Efficient allocation per Iteration']

OrderedDict([(0,
              OrderedDict([('Bidder_0',
                            {'good_ids': [0,
                              1,
                              2,
                              3,
                              4,
                              6,
                              7,
                              8,
                              10,
                              11,
                              12,
                              13,
                              14,
                              15,
                              16,
                              17],
                             'value': 392.0473444342457}),
                           ('Bidder_1', {'good_ids': [], 'value': 0}),
                           ('Bidder_2', {'good_ids': [], 'value': 0}),
                           ('Bidder_3', {'good_ids': [], 'value': 0}),
                           ('Bidder_4', {'good_ids': [], 'value': 0}),
                           ('Bidder_5', {'good_ids'

In [27]:
#LSVM
#MRVM
#GSVM
for i_load in results_all:
    mean_scores = []
    for res_l in i_load:
        try:
            mean_scores.append(res_l['MLCA Efficiency'])
            #print(res_l['MLCA Efficiency'])
        except:
            pass
    print(np.mean(mean_scores), np.std(mean_scores))

0.8398288549098961 0.05182647459019574
0.8031062808481103 0.040352068583338244
0.9337057142031697 0.04263073235248043
0.7507823537018439 0.07330770972253216
